![](ubc_header.png)

# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

****

## Part I: Data Preprocessing

## Set up and Import Libraries

In [1]:
#pip install -r requirements.txt

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest

In [3]:
# RUN ONLY ONCE
# os.chdir is used to change the current directory to the specified path
os.chdir("../") # Sets path to the repo folder as it is one level above where this file exists!
path = os.getcwd()
print(path)

/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024


****

## Load Data Files

### Set Data File Path

In [14]:
# Select data file path for the chosen venue and time range where the recipes data stored

# OK 2023
filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "OK 23-24 Sep-Dec*", "*.oc"))

# Totem 2023
# filepath_list += glob.glob(os.path.join(os.getcwd(), "data", "raw", "Feast 23-24 Sep-Dec*", "*.oc"))

# Gather 2023
# filepath_list += glob.glob(os.path.join(os.getcwd(), "data", "raw", "Gather 23-24 Sep-Dec*", "*.oc"))

filepath_list

['/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024/data/raw/OK 23-24 Sep-Dec/OK_ALL_prods_July29_2023.oc']

### Import Items List

In [15]:
# Read items.xml files in the filepath_list and construct a dataframe
ItemId = []
Description = []
CaseQty = []
CaseUOM = []
PakQty = []
PakUOM = []
InventoryGroup = []


# from the items xml file, findtext of CaseQty, CaseUOM, PakQty, PakUOM, and InventoryGroup
# then append it on the lists above

for filepath in filepath_list:
    path = filepath + '/items.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for item in xtree.iterfind('Item'):
            ItemId.append(item.attrib['id'])
            Description.append(item.findtext('Description'))
            CaseQty.append(item.findtext('CaseQty'))
            CaseUOM.append(item.findtext('CaseUOM'))
            PakQty.append(item.findtext('PakQty'))
            PakUOM.append(item.findtext('PakUOM'))
            InventoryGroup.append(item.findtext('InventoryGroup'))

            
# Create a dataframe from the lists created above.
        
Items = pd.DataFrame({'ItemId': ItemId, 'Description': Description, 'CaseQty': CaseQty, 
                      'CaseUOM': CaseUOM, 'PakQty': PakQty, 'PakUOM': PakUOM, 'InventoryGroup': InventoryGroup}
                    )
Items.drop_duplicates(inplace=True)

Items.reset_index(drop=True, inplace=True)

In [16]:
# creates a new array with unique ItemIds
all_id_list = Items["ItemId"].unique()

In [17]:
# None of the items are egg yolk liq
Items.loc[Items["Description"] == "Egg Yolk Liq"]

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


In [18]:
# Sumplemental option for egg, vegan option
Items.loc[Items["ItemId"] == "I-68700"]

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
434,I-68700,EGG CKD*FOLDED VEGAN FRZ,60.000,each,1.000,each,DAIRY


In [19]:
# Gives you the list of breads
breadlist = []


# If the items in the Items list is a "LOAF", "SANDWICH", or "BREAD" then we should add it in the breadlist
# We append it onto the breadlist at position row["ItemId"]
for ind, row in Items.iterrows():
    if ("LOAF" or "SANDWICH" "BREAD") in row["Description"]:
        breadlist.append(row["ItemId"])

breadlist

['I-47962',
 'I-1160',
 'I-52561',
 'I-13004',
 'I-1186',
 'I-1187',
 'I-64492',
 'I-1230',
 'I-1232',
 'I-1271',
 'I-14190',
 'I-11664',
 'I-24229',
 'I-24227',
 'I-15032',
 'I-12478',
 'I-1275',
 'I-11842']

In [20]:
# Based on info below there are 486 rows and 7 columns
Items.shape

(1341, 7)

In [21]:
Items.dtypes

ItemId            object
Description       object
CaseQty           object
CaseUOM           object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [22]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Items_List.csv")
Items.to_csv(path, index = False, header = True)

### Import Ingredients List

In [15]:
# Read ingredients.xml files in the filepath_list and construct a dataframe
IngredientId = []
Conversion = []
InvFactor = []
Qty = []
Recipe = []
Uom = []

# Using the Ingredients XML file, we extract attributes containing ingredients, conversion, invFactor, qty, recipe, and uom. 
# Then we append it onto the IngredientId, Coversion, InvFactor, Qty, Recipe, and Uom lists
# Then we create a dataframe using the lists created. 

for filepath in filepath_list:
    path = filepath + '/Ingredients.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Ingredient'):
            IngredientId.append(x.attrib['ingredient'])
            Conversion.append(x.attrib['conversion'])
            InvFactor.append(x.attrib['invFactor'])
            Qty.append(x.attrib['qty'])
            Recipe.append(x.attrib['recipe'])
            Uom.append(x.attrib['uom'])
    
Ingredients = pd.DataFrame({'IngredientId': IngredientId, 'Qty': Qty,'Uom': Uom, 'Conversion': Conversion, 
                      'InvFactor': InvFactor,'Recipe': Recipe}).drop_duplicates()
Ingredients.drop_duplicates(subset=["IngredientId", "Recipe"], inplace=True)

Ingredients.reset_index(drop=True, inplace=True)

In [17]:
Ingredients.loc[Ingredients["IngredientId"] == "I-29389"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
3690,I-29389,800.000,g,0.00100000,800.0000,P-50393
4125,I-29389,2.500,Kg,1.00000000,1.0000,P-50739
6622,I-29389,600.000,g,0.00100000,150.0000,P-57344
13099,I-29389,15.000,g,0.00100000,1.0000,R-56966
13103,I-29389,15.000,g,0.00100000,1.0000,R-56967


In [19]:
# This will output the IngredientId on the right side and the number of times the ingredient appears in the Ingredients
# dataframe on the left side. 

# The duplicated() method returns a Series with True and False values that describe which rows in the DataFrame are 
# duplicated and not.

check = Ingredients["IngredientId"].duplicated()

# The line below tells us at which index is the ingredient duplicated. For example I-4598 exists on index 2 of the dataframe and
# then again on index 8. So duplicate is first true on index 8, which is why it is printed below. It also appears in a below 
# index so it is printed again with that index number. 
Ingredients["IngredientId"][check]

24       P-26020
27        I-1813
33        I-3642
34       I-47774
40        I-5983
41        I-4598
46        I-6026
49        I-3388
50        I-3642
52        I-6026
57        I-3992
58        I-4589
60        I-4742
62        I-5983
64        I-3284
65        I-3660
66        I-3388
68        I-4919
73        I-5983
74        I-1813
77       P-18907
82        I-3388
83        I-3642
84        I-6026
85        I-3388
90        I-6026
93        I-4589
94        I-4598
96        I-3387
98        I-6006
120       I-5983
122       I-5983
124       I-5983
126       I-5983
128       I-5983
156       I-5983
158       I-4792
163       I-3388
165       I-5983
166       I-6026
167       I-6443
175       I-3321
182       I-6006
183      I-12176
185       I-3387
186       I-3643
188       I-3619
189       I-3649
190       I-3660
194       I-6026
198      P-18907
199       I-3642
200       I-6026
201       I-3643
202       I-6026
203       I-3388
206       I-5983
211       I-1813
213       I-32

In [20]:
# We can see from the printed items above that I-64877 does not appear because it does not get duplicated
Ingredients.loc[Ingredients["IngredientId"] == "I-64877"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
15811,I-64877,1.000,ea,1.00000000,1.0000,R-68698


In [23]:
Ingredients.loc[Ingredients["IngredientId"] == "I-64877"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
15811,I-64877,1.000,ea,1.00000000,1.0000,R-68698


In [24]:
# Below we are checking if we can extract an entire recipe. So we can do .loc with the particular recipe id and then 
# print all the data points (ingredientId, Qty, Uom, Conversion, InvFactor) for that recipe. 
Ingredients.loc[Ingredients["Recipe"] == "R-68698"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
15810,I-60772,30.000,ml,0.00100000,1.0000,R-68698
15811,I-64877,1.000,ea,1.00000000,1.0000,R-68698
15812,I-68700,0.500,each,1.00000000,1.0000,R-68698
15813,I-68718,1.000,each,1.00000000,1.0000,R-68698
15814,P-26069,10.000,g,1.00000000,1.0000,R-68698
15815,P-46862,30.000,g,0.00100000,1.0000,R-68698


In [25]:
Items.loc[Items["ItemId"] == "I-68700"]

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
434,I-68700,EGG CKD*FOLDED VEGAN FRZ,60.000,each,1.000,each,DAIRY


In [26]:
Ingredients.shape

(16050, 6)

In [27]:
Ingredients.dtypes

IngredientId    object
Qty             object
Uom             object
Conversion      object
InvFactor       object
Recipe          object
dtype: object

In [28]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Ingredients_List.csv")
Ingredients.to_csv(path, index = False, header = True)

### Import Preps List

In [29]:
# Read preps.xml files in the filepath_list and construct a dataframe
PrepId = []
Description = []
PakQty = []
PakUOM = []
InventoryGroup = []


# Here we do the same thing for the Preps XML file where we find the columns using attrib function and then append it onto
# the dataframe called Preps. 

for filepath in filepath_list:
    path = filepath + '/Preps.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Prep'):
            PrepId.append(x.attrib['id'])
            Description.append(x.findtext('Description'))
            PakQty.append(x.findtext('PakQty'))
            PakUOM.append(x.findtext('PakUOM'))
            InventoryGroup.append(x.findtext('InventoryGroup'))
    
Preps = pd.DataFrame({'PrepId': PrepId, 'Description': Description,
                  'PakQty': PakQty, 'PakUOM':PakUOM, 'InventoryGroup': InventoryGroup}).drop_duplicates()
preps_columns = Preps.columns
Preps.drop_duplicates(subset=["PrepId"], inplace=True)

Preps.reset_index(drop=True, inplace=True)

In [30]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-61322,AIOLI|Pesto,4.000,L,ZDONT USE OK - PREP
1,P-61009,ALF|Flatbread|Brisket,1.000,un,
2,P-26184,BAKED|Beans,9.000,Kg,PREP
3,P-53798,BAKED|Berry Crisp,45.000,Kg,
4,P-64511,Baked|Brocoli|Mushroom|Pasta,6.000,Kg,
5,P-64689,BAKED|Chicken Thigh|FNH,34.000,Kg,
6,P-58410,BAKED|Gnocchi,1.570,Kg,
7,P-26965,BAKED|Jalapeno Cornbread,12.000,piece,PREP
8,P-50502,BAKED|Meatball,310.000,each,PREP
9,P-54644,BAKED|Moussaka,16.000,PTN,


In [31]:
# This tells us that there are not any duplicates in the Preps dataframe. There is a different process|ingredient pair for 
# all the recipes. 
check = Preps["PrepId"].duplicated().any()
print(check)

False


In [32]:
# There are 546 rows and 5 columns
Preps.shape

(1742, 5)

In [33]:
# Here we see there is only one place P-50739 is used in the Preps
Preps.loc[Preps["PrepId"] == "P-50739"]

,PrepId,Description,PakQty,PakUOM,InventoryGroup
170,P-50739,COMPOTE|Apple Cinnamon,2.500,L,PREP


In [34]:
Preps.dtypes

PrepId            object
Description       object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [35]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Preps_List.csv")
Preps.to_csv(path, index = False, header = True)

### Import Products List

In [36]:
# Read products.xml files in the filepath_list and construct a dataframe
ProdId = []
Description = []
SalesGroup = []

# From the XML file for products append the id into ProdId, description into Description, and SalesGroup into SalesGroup list.
# Make a dataframe out of the 3 lists
# Then also drop the duplicates in the Products dataframe

for filepath in filepath_list:
    path = filepath + '/Products.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Prod'):
            ProdId.append(x.attrib['id'])
            Description.append(x.findtext('Description'))
            SalesGroup.append(x.findtext('SalesGroup'))
        
Products = pd.DataFrame({'ProdId': ProdId, 'Description': Description, 'SalesGroup': SalesGroup})
Products.drop_duplicates(inplace=True)

Products.reset_index(drop=True, inplace=True)

In [37]:
Products

,ProdId,Description,SalesGroup
0,R-67294,$1 OWN MUG ETHICAL BEAN COFFEE,BEVERAGE
1,R-62976,ADD CUP,OK - GRAB & GO
2,R-31006,ADD|Beef,OK - CUSTOM KITCHEN
3,R-31131,ADD|Chicken,OK - CUSTOM KITCHEN
4,R-30154,ADD|Crackers,OK - CUSTOM KITCHEN
5,R-41877,ADD|Fish,OK - GRILL KITCHEN DAY
6,R-32124,ADD|Guacamole,OK - SANDWICH KITCHEN
7,R-60997,ADD|Guacamole.,OK - SANDWICH KITCHEN
8,R-61591,ADD|Karaage Chicken|4oz,OK - GRILL KITCHEN FEATURES
9,R-41753,ADD|Salad,OK - BURRITO KITCHEN


In [38]:
# Here we can see that there is only one R-56966 in the products 
Products.loc[Products["ProdId"] == "R-56966"]

,ProdId,Description,SalesGroup
1934,R-56966,VEG|Oats|Steel Cut|Maple Apple,OK - VEGETARIAN KITCHEN


In [39]:
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [40]:
# Here we can see that there is only one R-68698 in the products 
Products.loc[Products["ProdId"] == "R-68698"]

,ProdId,Description,SalesGroup
847,R-68698,GRL|Vegan Breakfast Sandwich,OK - GRILL KITCHEN BREAKFAST


In [41]:
Products.shape

(2005, 3)

In [42]:
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [43]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Products_List.csv")
Products.to_csv(path, index = False, header = True)

### Import Conversions List

In [44]:
# Read conventions.xml files in the filepath_list and construct a dataframe
ConversionId = []
Multiplier = []
ConvertFromQty = []
ConvertFromUom = []
ConvertToQty = []
ConvertToUom = []

# From the XML file for Conversions append the id into ConversionId, multiplier into Multiplier, ConvertFrom->qty into 
# ConvertFromQty,ConvertFrom->uom into ConvertFromUom, ConvertTo->qty into ConvertToQty and and ConvertTo->uom into the
# CovertToUom list. 
# Make a dataframe out of the 3 lists
# Then also drop the duplicates in the Products dataframe


for filepath in filepath_list:
    path = filepath + '/Conversions.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Conversion'):
            ConversionId.append(x.attrib['id'])
            Multiplier.append(x.attrib['multiplier'])
            ConvertFromQty.append(x.find('ConvertFrom').attrib['qty'])
            ConvertFromUom.append(x.find('ConvertFrom').attrib['uom'])
            ConvertToQty.append(x.find('ConvertTo').attrib['qty'])
            ConvertToUom.append(x.find('ConvertTo').attrib['uom'])
    
    
Conversions = pd.DataFrame({'ConversionId': ConversionId, 'Multiplier': Multiplier, 'ConvertFromQty': ConvertFromQty,
                           'ConvertFromUom': ConvertFromUom, 'ConvertToQty': ConvertToQty, 'ConvertToUom': ConvertToUom}
                          ).drop_duplicates()

Conversions.reset_index(drop=True, inplace=True)

In [45]:
# Here we can see for example that to convert 1.14 L to 1 L the multiplier is 0.877 since 1/1.14 = 0.877
Conversions

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,,1.00000000,1.0000,XXX,1.0000,L
1,,0.87719298,1.0000,1.14L,1.1400,L
2,,0.66666667,1.0000,1.5L,1.5000,L
3,,0.57142857,1.0000,1.75 L,1.7500,L
4,,0.50000000,1.0000,2L,2.0000,L
5,,0.25000000,1.0000,4L,4.0000,L
6,,0.08333333,1.0000,FOOT,12.0000,INCH
7,,0.06250000,1.0000,16L,16.0000,L
8,,0.05917160,1.0000,1/2LTR,16.9000,fl oz
9,,0.03937008,1.0000,750ML,25.4000,fl oz


In [46]:
# Here we can check that there are no ingredients listed since 
Conversions.loc[Conversions["ConversionId"] == "I-4582"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [47]:
all_id_list = Items["ItemId"].unique()
all_conv_list = Conversions["ConversionId"].unique()

print("All unique IDs list\n")
print(all_id_list)
print("\n")
print("All unique Conversions list\n")
print(all_conv_list)

All unique IDs list

['I-7631' 'I-2353' 'I-43239' ... 'I-1489' 'I-47845' 'I-4967']


All unique Conversions list

['' 'Iqq-1093' 'I-1100' 'I-1120' 'I-1195' 'I-1197' 'I-1232' 'I-1277'
 'I-1791' 'I-1813' 'I-1814' 'I-1815' 'I-1820' 'I-1821' 'I-1827' 'I-2074'
 'I-2159' 'I-2205' 'I-2207' 'I-2217' 'I-2218' 'I-2220' 'I-2221' 'I-2236'
 'I-2244' 'I-2248' 'I-2251' 'I-2252' 'I-2254' 'I-2306' 'I-2310' 'I-2933'
 'I-3074' 'I-3113' 'I-3141' 'I-3143' 'I-3144' 'I-3145' 'I-3146' 'I-3149'
 'I-3159' 'I-3223' 'I-3224' 'I-3225' 'I-3229' 'I-3237' 'I-3284' 'I-3321'
 'I-3348' 'I-3356' 'I-3365' 'I-3367' 'I-3370' 'I-3386' 'I-3387' 'I-3388'
 'I-3389' 'I-3390' 'I-3391' 'I-3392' 'I-3394' 'I-3435' 'I-3449' 'I-3458'
 'I-3468' 'I-3497' 'I-3501' 'I-3517' 'I-3545' 'I-3563' 'I-3572' 'I-3575'
 'I-3577' 'I-3582' 'I-3584' 'I-3589' 'I-3590' 'I-3591' 'I-3617' 'I-3619'
 'I-3620' 'I-3621' 'I-3622' 'I-3623' 'I-3624' 'I-3625' 'I-3628' 'I-3629'
 'I-3630' 'I-3632' 'I-3633' 'I-3634' 'I-3635' 'I-3636' 'I-3637' 'I-3638'
 'I-3639' 'I-3

In [48]:
# Here we have the number of items in the list that are a part of the "all_id_list" but not part of the "all_conv_list"
missing_conv_id = []

for item in all_id_list:
    if item not in all_conv_list:
        missing_conv_id.append(item)
        
missing_conv_id
print(len(missing_conv_id))

1103


In [49]:
Conversions.shape

(444, 6)

In [50]:
Conversions.loc[Conversions["ConversionId"] == "I-29389"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [51]:
Conversions.dtypes

ConversionId      object
Multiplier        object
ConvertFromQty    object
ConvertFromUom    object
ConvertToQty      object
ConvertToUom      object
dtype: object

In [52]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Conversions_List.csv")
Conversions.to_csv(path, index = False, header = True)

***
## Data Summary

In [53]:
# Summary of raw data imported for evaluation
# Here we have a summary of the number of items, preps, ingredients, products, conversions

datasum = pd.DataFrame([Items.shape, Preps.shape, Ingredients.shape, Products.shape, Conversions.shape],
                       columns = ['count', 'columns'], 
                       index = ['Items', 'Preps', 'Ingredients', 'Products', 'Conversions'])
datasum

,count,columns
Items,1341,7
Preps,1742,5
Ingredients,16050,6
Products,2005,3
Conversions,444,6
